# DONE:

- x include non-Metaphor data and label

- repeat experiments after sampling original sentences, not random sampling

- fix the damn indentation on the accuracy

- Try freezing some layers. Hopefully reduces training time (significantly)

- FIXING BERT TOKENIZER TO REDEWIEDERGABE!!!

- make use of cuda or mps nicer

- increase freeze rate (Dropped performance slightly)

- add additional dropout layer (not really sure if it helped)

# NOW:

- include loss and score plots

# TODO:
    
- include F1-score in evaluation

- x test F1-score loss function
    - https://www.kaggle.com/code/bigironsphere/loss-function-library-keras-pytorch/notebook
    - not avaiable for mps yet!


In [1]:
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
import seaborn as sns

import os
import re

np.random.seed(3093453)
sns.set(context="talk", style="darkgrid")

In [2]:
cwd = os.getcwd()
folder = "clean+translated-data"
file = "clean-de.csv"
path = "/".join( (cwd, folder, file) )

df = pd.read_csv(path, index_col=0)
df["lang"] = "de"

files = os.listdir(folder)
files = [file for file in files if re.match(r'clean-[a-z]{2}-de.csv', file)]

for file in files:
    path = "/".join( (cwd, folder, file) )
    data = pd.read_csv(path, index_col=0)
    data["lang"] = file[6:8]
    
    df = pd.concat([df, data])

del(data)
df.reset_index(inplace=True)
df

,index,Textstelle,Metapher,Kandidat,Nein,Metapher?,Fokus,Rahmen,Annotator,orig,lang
0,0,Der politische Körper verwendet in beiden Fäll...,3,1,0,Metapher,"['Zähnen und Krallen', 'Zähnen und Krallen', n...","['der politische Körper', 'der politische Körp...","['B', 'K', 'A', 'GoldStandard']","""Der politische Körper verwendet in beiden Fä...",de
1,1,Immer noch wird durch die protestantische Lehr...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Immer noch wird durch die protestantische Le...",de
2,2,"Kein Zweifel, schreibt Alb. Schaeffle, vorauss...",0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Kein Zweifel"", schreibt Alb. Schaeffle 1), v...",de
3,3,Mit je weiterem Blick wir die Stoffwelt zu übe...,1,3,0,Metaphernkandidat,"[nan, 'Stoffwelt, verzwergt und entkleidet', n...","[nan, 'unser praktisches Ideal', nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Mit je weiterem Blick wir die Stoffwelt zu ü...",de
4,4,Solange das Menschengeschlecht seinen Zerstöre...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Solange das Menschengeschlecht seinen Zerstö...",de
...,...,...,...,...,...,...,...,...,...,...,...
3990,794,Sie hat durchweg Naturwissenschaft in dem von ...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['T', 'P', 'B', 'GoldStandard']",verlaufs; sie hat Naturforschung in dem von un...,da
3991,795,Wenn wir uns ein Prinzip ausdenken und darauf ...,1,3,0,Metaphernkandidat,"[nan, nan, 'verstellbare Puppen', nan]","[nan, nan, 'staatliche Menschenverhältnisse', ...","['P', 'K', 'B', 'GoldStandard']",wenn wir uns ein Prinzip denken und auf Grund ...,da
3992,796,sieht man von dem Schimpfwort Rechtsphilosophi...,3,1,0,Metapher,"['schwindsüchtig ist', nan, 'schwindsüchtig', ...","['daß eine Philosophie', nan, 'eine Philosophi...","['P', 'K', 'B', 'GoldStandard']",wenn wir von dem schlechten Worte »Rechtsphilo...,da
3993,797,"Die Macht der Regierung, die verschiedenen Mot...",0,2,0,NaN,"[nan, nan]","[nan, nan]","['P', 'B']","»Die Macht, die das Kabinett hat, auf die vers...",da


In [3]:
cwd = os.getcwd()
folder = "clean+translated-data"
file = "non-metaphor-clean-de.csv"
path = "/".join( (cwd, folder, file) )

non = pd.read_csv(path)
non["lang"] = "de"
non

,index,Textstelle,Metapher,Kandidat,Nein,Metapher?,Fokus,Rahmen,Annotator,orig,lang
0,0,Eine sammlung von preisscliristenz Herausgegeb...,0,0,1,Nein,NaN,NaN,X,Eine sammlung von preisscliristenz Herausgegeb...,de
1,1,Januar veröffentlichten die Professoren Haeck...,0,0,1,Nein,NaN,NaN,X,Januar 1900 veröffentlichten die Professoren H...,de
2,2,"Unter den Abhandlungen und Schriften, welche ...",0,0,1,Nein,NaN,NaN,X,"Unter den 60 Abhandlungen und Schriften, welch...",de
3,3,In dem vorliegenden Sammelwerke werden sieben ...,0,0,1,Nein,NaN,NaN,X,In dem vorliegenden Sammelwerke werden sieben ...,de
4,4,"Es folgt hier zunächst das Preisausschreiben, ...",0,0,1,Nein,NaN,NaN,X,"Es folgt hier zunächst das Preisausschreiben, ...",de
...,...,...,...,...,...,...,...,...,...,...,...
3960,3960,In der Zeit des Humanismus und der Reformation...,0,0,1,Nein,NaN,NaN,X,In der Zeit des Humanismus und der Reformation...,de
3961,3961,Den Mittelpunkt der Bildung stellten die alten...,0,0,1,Nein,NaN,NaN,X,Den Mittelpunkt der Bildung stellten die alten...,de
3962,3962,Jahrhunderts hat eine beständige Differenzieru...,0,0,1,Nein,NaN,NaN,X,Jahrhunderts hat eine beständige Differenzieru...,de
3963,3963,Die Pädagogik der Neuzeit scheint davon wenig ...,0,0,1,Nein,NaN,NaN,X,Die Pädagogik der Neuzeit scheint davon wenig ...,de


In [4]:
df = pd.concat([df, non])
df.reset_index(drop=True)
df.rename(columns= {"Kandidat": "Metaphernkandidat"}, inplace=True)
df

,index,Textstelle,Metapher,Metaphernkandidat,Nein,Metapher?,Fokus,Rahmen,Annotator,orig,lang
0,0,Der politische Körper verwendet in beiden Fäll...,3,1,0,Metapher,"['Zähnen und Krallen', 'Zähnen und Krallen', n...","['der politische Körper', 'der politische Körp...","['B', 'K', 'A', 'GoldStandard']","""Der politische Körper verwendet in beiden Fä...",de
1,1,Immer noch wird durch die protestantische Lehr...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Immer noch wird durch die protestantische Le...",de
2,2,"Kein Zweifel, schreibt Alb. Schaeffle, vorauss...",0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Kein Zweifel"", schreibt Alb. Schaeffle 1), v...",de
3,3,Mit je weiterem Blick wir die Stoffwelt zu übe...,1,3,0,Metaphernkandidat,"[nan, 'Stoffwelt, verzwergt und entkleidet', n...","[nan, 'unser praktisches Ideal', nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Mit je weiterem Blick wir die Stoffwelt zu ü...",de
4,4,Solange das Menschengeschlecht seinen Zerstöre...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Solange das Menschengeschlecht seinen Zerstö...",de
...,...,...,...,...,...,...,...,...,...,...,...
3960,3960,In der Zeit des Humanismus und der Reformation...,0,0,1,Nein,NaN,NaN,X,In der Zeit des Humanismus und der Reformation...,de
3961,3961,Den Mittelpunkt der Bildung stellten die alten...,0,0,1,Nein,NaN,NaN,X,Den Mittelpunkt der Bildung stellten die alten...,de
3962,3962,Jahrhunderts hat eine beständige Differenzieru...,0,0,1,Nein,NaN,NaN,X,Jahrhunderts hat eine beständige Differenzieru...,de
3963,3963,Die Pädagogik der Neuzeit scheint davon wenig ...,0,0,1,Nein,NaN,NaN,X,Die Pädagogik der Neuzeit scheint davon wenig ...,de


## Sentence ID

We need to do this to make sure we dont have a german sentence in the train data
... and a english version in the test data, which skewes the results

(Essentially the model would just memorize seen points, rather than learn what a metaphor is)



In [5]:
labs = ["Metapher", "Metaphernkandidat", "Nein"]
labels = dict(zip(range(3), labs))

# impute missing labels with silver labels
missing_lab = df["Metapher?"].isnull()
df.loc[missing_lab, "Metapher?"] = [labels[i] for i in df.loc[missing_lab, labs].to_numpy().argmax(axis=1)]

In [6]:
unique_sentences = df.orig.unique()

sentence_id = [np.where(sentence == unique_sentences)[0].item() for sentence in df.orig]
df["sentence_id"] = sentence_id

In [7]:
df

,index,Textstelle,Metapher,Metaphernkandidat,Nein,Metapher?,Fokus,Rahmen,Annotator,orig,lang,sentence_id
0,0,Der politische Körper verwendet in beiden Fäll...,3,1,0,Metapher,"['Zähnen und Krallen', 'Zähnen und Krallen', n...","['der politische Körper', 'der politische Körp...","['B', 'K', 'A', 'GoldStandard']","""Der politische Körper verwendet in beiden Fä...",de,0
1,1,Immer noch wird durch die protestantische Lehr...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Immer noch wird durch die protestantische Le...",de,1
2,2,"Kein Zweifel, schreibt Alb. Schaeffle, vorauss...",0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Kein Zweifel"", schreibt Alb. Schaeffle 1), v...",de,2
3,3,Mit je weiterem Blick wir die Stoffwelt zu übe...,1,3,0,Metaphernkandidat,"[nan, 'Stoffwelt, verzwergt und entkleidet', n...","[nan, 'unser praktisches Ideal', nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Mit je weiterem Blick wir die Stoffwelt zu ü...",de,3
4,4,Solange das Menschengeschlecht seinen Zerstöre...,0,4,0,Metaphernkandidat,"[nan, nan, nan, nan]","[nan, nan, nan, nan]","['B', 'K', 'A', 'GoldStandard']","""Solange das Menschengeschlecht seinen Zerstö...",de,4
...,...,...,...,...,...,...,...,...,...,...,...,...
3960,3960,In der Zeit des Humanismus und der Reformation...,0,0,1,Nein,NaN,NaN,X,In der Zeit des Humanismus und der Reformation...,de,4759
3961,3961,Den Mittelpunkt der Bildung stellten die alten...,0,0,1,Nein,NaN,NaN,X,Den Mittelpunkt der Bildung stellten die alten...,de,4760
3962,3962,Jahrhunderts hat eine beständige Differenzieru...,0,0,1,Nein,NaN,NaN,X,Jahrhunderts hat eine beständige Differenzieru...,de,4761
3963,3963,Die Pädagogik der Neuzeit scheint davon wenig ...,0,0,1,Nein,NaN,NaN,X,Die Pädagogik der Neuzeit scheint davon wenig ...,de,4762


In [8]:
file = "total.csv"
path = "/".join( (cwd, folder, file) )
df.to_csv(path)